We implement the analytical calculations done using Transfer Matrix formalism in the paper for the exact blocking of 1d Ising Model, and calculate $J$ and $h$ for several RG steps.

In [ ]:
using LinearAlgebra
using Plots
cs = cgrad(:RdBu)
# ----------------------------------------------------
# Set precision for BigFloat
# ----------------------------------------------------
setprecision(256)

# ----------------------------------------------------
# Define functions
# ----------------------------------------------------

# Function to construct the partition function transfer matrix Z with BigFloat
function transfer_matrix_Z(J::BigFloat, h::BigFloat)
    S = [
        exp(J + h)  exp(-J)
        exp(-J)     exp(J - h)
    ]
    return S
end

# Function to compute the trace of S^N using BigFloat
function compute_trace_of_power(S::Matrix{BigFloat}, N::Int)
    return tr(S^N)
end

# Function to construct the transfer matrix T based on sigma_i with BigFloat
function transfer_matrix(J::BigFloat, h::BigFloat, sigma_i::Int)
    factor = BigFloat(1) / BigFloat(2)
    if sigma_i == 1
        T = [
            (BigFloat(2) * exp(BigFloat(2) * h + BigFloat(2) * J) + exp(-BigFloat(2) * J))   (BigFloat(2) * exp(BigFloat(2) * h) + BigFloat(1))
            BigFloat(1)                                                                    exp(-BigFloat(2) * J)
        ]
    elseif sigma_i == -1
        T = [
            exp(-BigFloat(2) * J)                             BigFloat(1)
            (BigFloat(2) * exp(-BigFloat(2) * h) + BigFloat(1)) (BigFloat(2) * exp(BigFloat(2) * J - BigFloat(2) * h) + exp(-BigFloat(2) * J))
        ]
    else
        error("Invalid value for sigma_i: $sigma_i. Must be either 1 or -1.")
    end
    return factor .* T
end

# Function to compute the cyclic trace for a given configuration of sigmas using BigFloat
function compute_cyclic_trace(J::BigFloat, h::BigFloat, sigmas::Vector{Int})
    n = length(sigmas)
    
    # Compute the partition function Z = tr(S^(2n))
    S = transfer_matrix_Z(J, h)
    Z = compute_trace_of_power(S, 2n)

    
    # Initialize the transfer matrix product with the last sigma (for cyclicity)
    T = transfer_matrix(J, h, sigmas[end])
    
    # Multiply the transfer matrices for the rest of the spins
    for i in 1:(n-1)
        T_next = transfer_matrix(J, h, sigmas[i])
        T = T * T_next
    end
    
    return tr(T) / Z
end

# Function to compute specific traces for different spin configurations using BigFloat
function compute_specific_traces(J::BigFloat, h::BigFloat, n::Int)
    # Case 1: All sigma values are +1
    sigmas_1 = fill(1, n)
    t_1 = compute_cyclic_trace(J, h, sigmas_1)
    
    # Case 2: All sigma values are -1
    sigmas_2 = fill(-1, n)
    t_2 = compute_cyclic_trace(J, h, sigmas_2)
    
    # Case 3: First sigma is -1, all others are +1
    sigmas_3 = vcat([-1], fill(1, n - 1))
    t_3 = compute_cyclic_trace(J, h, sigmas_3)
    
    return t_1, t_2, t_3
end

# ----------------------------------------------------
# RG Iterations
# ----------------------------------------------------

# Initial values
J = BigFloat("0.35")   # Example coupling constant
h = BigFloat("0.17")   # Example external field
rg_steps = 4
N_sites = 2^8          # Keep as an integer

# Arrays to store J and h in BigFloat form
J_values = [J]
h_values = [h]

# Perform the RG iterations
for i in 1:rg_steps
    println("\nIteration $i:")
    J_round = round(Float64(J), digits=3)
    h_round = round(Float64(h), digits=3)
    println("J and h are currently $J_round and $h_round")
    
    # Compute traces for the three specific spin configurations
    t_1, t_2, t_3 = compute_specific_traces(J, h, div(N_sites, 2))
    
    # Check for zeros or negative values
    if t_1 <= 0 || t_2 <= 0 || t_3 <= 0
        error("One of the traces is zero or negative, which will cause log() to fail.")
    end
    
    #println("Traces are t1 = $t_1, t2 = $t_2, t_3 = $t_3")
    
    #Compute J_new and h_new using the normalized probabilities
    J_new = (BigFloat(1)/BigFloat(4)) * (log(t_1) - log(t_3)) - (BigFloat(1)/(BigFloat(2) * BigFloat(N_sites))) * (-log(t_2) + log(t_1))
    h_new = (BigFloat(1)/BigFloat(N_sites)) * (log(t_1) - log(t_2))

    push!(J_values, J_new)
    push!(h_values, h_new)
    
    # Update for next iteration
    J = J_new
    h = h_new
    N_sites = div(N_sites, 2)
end

# Convert J_values and h_values to Float64 for plotting
J_values_float = Float64.(J_values)
h_values_float = Float64.(h_values)

# Define the index for the RG steps (0 through rg_steps)
rg_steps_indices = 0:rg_steps

# ----------------------------------------------------
# "Old"/estimated J and h values
# ----------------------------------------------------
#Data from ML procedure
J_est = [0.35, 0.1507117061891995, 0.04553711766025867, 0.011296427064370871, 0.002762435413881227]
h_est = [0.17, 0.25275486605037156, 0.3102550153548688, 0.3312916492199335, 0.3365380765581261]

# For those arrays, we assume they correspond to 5 steps: indices 0:4
est_steps_indices = 0:(length(J_est)-1)  # 0:4

# ----------------------------------------------------
# Combined Plots
# ----------------------------------------------------
palette = cgrad(:GnBu, 6)

# Extract the middle two colors
middle_color1 = palette[3]
middle_color2 = palette[4]

# 1) Combined Plot for J
p1 = plot(
    est_steps_indices,
    J_est,
    label = "Estimated J",
    xlabel = "RG Step",
    ylabel = "Coupling J",
    linewidth = 2,
    marker = :circle,
    legend = :topright,
    color = cs[2]
)

plot!(
    rg_steps_indices,
    J_values_float,
    label     = "RG-Evolved J",
    linewidth = 2,
    marker    = :s,
    color = :pink)
savefig(p1, "combined_J_evolution.pdf")


# 2) Combined Plot for h
p2 = plot(
    est_steps_indices,
    h_est,
    label = "Estimated h",
    xlabel = "RG Step",
    ylabel = "External Field h",
    linewidth = 2,
    marker = :circle,
    legend = :bottomright,
    color = cs[end-1]
)

plot!(
    rg_steps_indices,
    h_values_float,
    label     = "RG-Evolved h",
    linewidth = 2,
    marker    = :s,
    color = cs[end-3]
)

savefig(p2, "combined_h_evolution.pdf")


# ----------------------------------------------------
# (Optional) Display the plots in the current session
# ----------------------------------------------------
display(p1)
display(p2)
